<div align = "center">

# **Exploración de Base Final Anual**

</div>

## Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Carga de Datos

In [3]:
# Cargar base final
df = pd.read_csv('../data/tmp/base_final_anual.csv')

print(f"Dimensiones: {df.shape}")
print(f"\nColumnas:")
print(df.dtypes)
df.head()

Dimensiones: (738902, 21)

Columnas:
ID_POLIZA_AMPARO_PERIODO        object
POLIZA                           int64
AMPARO                          object
ANIO_POLIZA                      int64
PERIODO                          int64
VIGENCIA_PERIODO_INICIO         object
VIGENCIA_PERIODO_FIN            object
DIAS_EXPOSICION                  int64
FRACCION_ANUAL                 float64
EXPOSICION_PRORRATEADA         float64
VIGENCIA_ORIGINAL_ID            object
COD_SUCURSAL                     int64
VIGENCIA_ORIGINAL_INICIO        object
VIGENCIA_ORIGINAL_FIN           object
DEPARTAMENTO_SINIESTRO          object
FECHA_DE_SINIESTRO              object
FECHA_AVISO                     object
SEVERIDAD                      float64
SEVERIDAD_EXTRAPOLADA_ANUAL    float64
TUVO_SINIESTRO                   int64
ES_PERIODO_ORIGINAL               bool
dtype: object


,ID_POLIZA_AMPARO_PERIODO,POLIZA,AMPARO,ANIO_POLIZA,PERIODO,VIGENCIA_PERIODO_INICIO,VIGENCIA_PERIODO_FIN,DIAS_EXPOSICION,FRACCION_ANUAL,EXPOSICION_PRORRATEADA,...,COD_SUCURSAL,VIGENCIA_ORIGINAL_INICIO,VIGENCIA_ORIGINAL_FIN,DEPARTAMENTO_SINIESTRO,FECHA_DE_SINIESTRO,FECHA_AVISO,SEVERIDAD,SEVERIDAD_EXTRAPOLADA_ANUAL,TUVO_SINIESTRO,ES_PERIODO_ORIGINAL
0,1020472_PRESTACIONES SOCIALES_2020_0_20200401,1020472,PRESTACIONES SOCIALES,2020,0,2020-04-01,2021-04-01,366,1.00274,1.402544e+07,...,12,01/04/2020,01/03/2027,13-BOLIVAR,30/04/2020,26/02/2024,6864750.0,6.845994e+06,1,True
1,1020472_PRESTACIONES SOCIALES_2021_1_20210402,1020472,PRESTACIONES SOCIALES,2021,1,2021-04-02,2022-04-02,366,1.00274,1.402544e+07,...,12,01/04/2020,01/03/2027,13-BOLIVAR,30/04/2020,26/02/2024,0.0,0.000000e+00,1,False
2,1020472_PRESTACIONES SOCIALES_2022_2_20220403,1020472,PRESTACIONES SOCIALES,2022,2,2022-04-03,2023-04-03,366,1.00274,1.402544e+07,...,12,01/04/2020,01/03/2027,13-BOLIVAR,30/04/2020,26/02/2024,0.0,0.000000e+00,1,False
3,1020472_PRESTACIONES SOCIALES_2023_3_20230404,1020472,PRESTACIONES SOCIALES,2023,3,2023-04-04,2024-04-03,366,1.00274,1.402544e+07,...,12,01/04/2020,01/03/2027,13-BOLIVAR,30/04/2020,26/02/2024,0.0,0.000000e+00,1,False
4,1020472_PRESTACIONES SOCIALES_2024_4_20240404,1020472,PRESTACIONES SOCIALES,2024,4,2024-04-04,2025-04-04,366,1.00274,1.402544e+07,...,12,01/04/2020,01/03/2027,13-BOLIVAR,30/04/2020,26/02/2024,0.0,0.000000e+00,1,False


## Exploración General

In [4]:
print("=" * 80)
print("RESUMEN ESTADÍSTICO GENERAL")
print("=" * 80)

print(f"\nTotal de registros: {len(df):,}")
print(f"Pólizas únicas: {df['POLIZA'].nunique():,}")
print(f"Amparos únicos: {df['AMPARO'].nunique()}")

print(f"\nAmparos disponibles:")
print(df['AMPARO'].value_counts())

RESUMEN ESTADÍSTICO GENERAL

Total de registros: 738,902
Pólizas únicas: 212,191
Amparos únicos: 15

Amparos disponibles:
AMPARO
PRESTACIONES SOCIALES                363085
ESTABILIDAD DE LA OBRA               128865
CALIDAD DEL SERVICIO                  91450
CUMPLIMIENTO                          56556
CALIDAD DE LOS ELEMENTOS              32261
CALIDAD Y CORRECTO FUNCIONAMIENTO     21477
SERIEDAD DE LA OFERTA                 18189
CALIDAD                               14338
BUEN MANEJO DEL ANTICIPO               6442
CORRECTO FUNCIONAMIENTO                2500
PROVISION DE REPUESTOS                 1545
PAGO ANTICIPADO                        1046
DISPOSICIONES LEGALES                   602
SUMINISTRO DE REPUESTOS                 443
BUEN MANEJO DE MATERIALES               103
Name: count, dtype: int64


## Análisis por Amparo (Cobertura)

In [ ]:
print("=" * 80)
print("ESTADÍSTICAS POR AMPARO")
print("=" * 80)

# Métricas por amparo
stats_amparo = df.groupby('AMPARO').agg({
    'POLIZA': 'count',  # Número de períodos
    'TUVO_SINIESTRO': ['sum', 'mean'],  # Siniestros y frecuencia
    'EXPOSICION_PRORRATEADA': 'sum',  # Exposición total
    'SEVERIDAD': ['sum', 'mean', 'std']  # Severidad
}).round(4)

stats_amparo.columns = ['N_PERIODOS', 'N_SINIESTROS', 'FRECUENCIA', 
                        'EXPOSICION_TOTAL', 'SEVERIDAD_TOTAL', 
                        'SEVERIDAD_MEDIA', 'SEVERIDAD_STD']

# Calcular prima pura = severidad_total / exposicion_total
stats_amparo['PRIMA_PURA'] = stats_amparo['SEVERIDAD_TOTAL'] / stats_amparo['EXPOSICION_TOTAL']

stats_amparo = stats_amparo.sort_values('N_PERIODOS', ascending=False)
stats_amparo

In [ ]:
# Visualización de frecuencia por amparo
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Frecuencia
stats_amparo['FRECUENCIA'].sort_values().plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('Frecuencia de Siniestros por Amparo')
axes[0].set_xlabel('Frecuencia (siniestros/período)')

# Exposición
stats_amparo['EXPOSICION_TOTAL'].sort_values().plot(kind='barh', ax=axes[1], color='darkgreen')
axes[1].set_title('Exposición Total por Amparo')
axes[1].set_xlabel('Exposición ($)')

plt.tight_layout()
plt.show()

## Clustering Jerárquico de Coberturas

In [ ]:
print("=" * 80)
print("CLUSTERING DE COBERTURAS POR COMPORTAMIENTO")
print("=" * 80)

# Preparar features para clustering
features_cluster = stats_amparo[['FRECUENCIA', 'SEVERIDAD_MEDIA', 'PRIMA_PURA', 'N_PERIODOS']].copy()
features_cluster = features_cluster.fillna(0)

# Normalizar
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_cluster)

print(f"\nFeatures para clustering:")
print(features_cluster)

### Dendrograma y Estructura Jerárquica

In [ ]:
print("=" * 80)
print("HIERARCHICAL CLUSTERING - DENDROGRAMA")
print("=" * 80)

# Calcular matriz de linkage (método Ward minimiza varianza intra-cluster)
linkage_matrix = linkage(features_scaled, method='ward')

# Dendrograma con líneas de corte para diferentes k
fig, ax = plt.subplots(figsize=(16, 10))

dend = dendrogram(
    linkage_matrix,
    labels=features_cluster.index.tolist(),
    leaf_rotation=45,
    leaf_font_size=9,
    ax=ax,
    color_threshold=0  # Colorear por defecto
)

# Agregar líneas horizontales para diferentes cortes
# Obtener alturas de corte para k=2,3,4,5
heights = sorted(linkage_matrix[:, 2], reverse=True)
colors = ['red', 'orange', 'green', 'blue']
k_values = [2, 3, 4, 5]

for i, k in enumerate(k_values):
    if k-1 < len(heights):
        h = heights[k-2] if k > 1 else heights[0]
        ax.axhline(y=h, color=colors[i], linestyle='--', alpha=0.7, 
                   label=f'k={k} (altura={h:.2f})')

ax.set_title('Dendrograma de Amparos (Método Ward)\nLíneas de corte para diferentes k', fontsize=14)
ax.set_xlabel('Amparo', fontsize=12)
ax.set_ylabel('Distancia (Ward)', fontsize=12)
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

print("\nInterpretación del dendrograma:")
print("- Altura del corte indica disimilitud entre clusters fusionados")
print("- Cortes más altos → clusters más diferentes entre sí")
print("- Método Ward minimiza la varianza dentro de cada cluster")

In [ ]:
print("=" * 80)
print("EVALUACIÓN DE NÚMERO ÓPTIMO DE CLUSTERS")
print("=" * 80)

# Calcular métricas para k = 2, 3, 4, 5
k_values = [2, 3, 4, 5]
silhouette_scores = []
cluster_assignments = {}

for k in k_values:
    agg = AgglomerativeClustering(n_clusters=k, linkage='ward')
    labels = agg.fit_predict(features_scaled)
    cluster_assignments[k] = labels
    sil_score = silhouette_score(features_scaled, labels)
    silhouette_scores.append(sil_score)
    print(f"k={k}: Silhouette Score = {sil_score:.4f}")

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Silhouette scores
axes[0].bar(k_values, silhouette_scores, color='steelblue', edgecolor='black')
axes[0].set_xlabel('Número de Clusters (k)')
axes[0].set_ylabel('Silhouette Score')
axes[0].set_title('Silhouette Score por Número de Clusters')
axes[0].set_xticks(k_values)

# Marcar el mejor
best_k = k_values[np.argmax(silhouette_scores)]
best_score = max(silhouette_scores)
axes[0].bar(best_k, best_score, color='darkgreen', edgecolor='black', label=f'Mejor: k={best_k}')
axes[0].legend()

# Distancias de fusión (para método del codo en jerárquico)
last_merges = linkage_matrix[-10:, 2]  # Últimas 10 fusiones
axes[1].plot(range(1, len(last_merges)+1), last_merges[::-1], 'bo-', markersize=8)
axes[1].set_xlabel('Número de Clusters')
axes[1].set_ylabel('Distancia de Fusión')
axes[1].set_title('Distancias de Fusión (Método del Codo Jerárquico)')
axes[1].set_xticks(range(1, len(last_merges)+1))

plt.tight_layout()
plt.show()

print(f"\n→ Mejor k según Silhouette Score: {best_k} (score = {best_score:.4f})")

### Resultados para k = 2, 3, 4, 5

In [ ]:
print("=" * 80)
print("ASIGNACIÓN DE CLUSTERS PARA k = 2, 3, 4, 5")
print("=" * 80)

# Crear DataFrame con todas las asignaciones
resultados_clustering = features_cluster.copy()
for k in k_values:
    resultados_clustering[f'CLUSTER_k{k}'] = cluster_assignments[k]

# Agregar métricas originales
resultados_clustering = resultados_clustering.join(stats_amparo[['N_SINIESTROS', 'EXPOSICION_TOTAL', 'SEVERIDAD_TOTAL']])

# Mostrar asignaciones para cada k
for k in k_values:
    print(f"\n{'='*60}")
    print(f"k = {k} CLUSTERS")
    print(f"{'='*60}")
    
    col_cluster = f'CLUSTER_k{k}'
    
    for cluster in range(k):
        mask = resultados_clustering[col_cluster] == cluster
        amparos_cluster = resultados_clustering[mask].index.tolist()
        
        # Estadísticas del cluster
        freq_media = resultados_clustering.loc[mask, 'FRECUENCIA'].mean()
        sev_media = resultados_clustering.loc[mask, 'SEVERIDAD_MEDIA'].mean()
        n_periodos = resultados_clustering.loc[mask, 'N_PERIODOS'].sum()
        n_siniestros = resultados_clustering.loc[mask, 'N_SINIESTROS'].sum()
        
        print(f"\nCluster {cluster} ({len(amparos_cluster)} amparos):")
        print(f"  Freq. media: {freq_media:.4f} | Sev. media: ${sev_media:,.0f}")
        print(f"  N períodos: {n_periodos:,} | N siniestros: {n_siniestros:,.0f}")
        print(f"  Amparos:")
        for amparo in amparos_cluster:
            freq = resultados_clustering.loc[amparo, 'FRECUENCIA']
            n = resultados_clustering.loc[amparo, 'N_PERIODOS']
            print(f"    • {amparo} (freq={freq:.4f}, n={n:,})")

In [ ]:
# Visualización comparativa de clusters para k = 2, 3, 4, 5
fig, axes = plt.subplots(2, 2, figsize=(16, 14))
axes = axes.flatten()

for idx, k in enumerate(k_values):
    ax = axes[idx]
    col_cluster = f'CLUSTER_k{k}'
    
    # Scatter plot
    scatter = ax.scatter(
        resultados_clustering['FRECUENCIA'],
        resultados_clustering['SEVERIDAD_MEDIA'],
        c=resultados_clustering[col_cluster],
        s=resultados_clustering['N_PERIODOS'] / resultados_clustering['N_PERIODOS'].max() * 400 + 80,
        cmap='viridis',
        alpha=0.7,
        edgecolors='black',
        linewidths=1.5
    )
    
    # Etiquetas
    for amparo in resultados_clustering.index:
        ax.annotate(
            amparo[:15] + '...' if len(amparo) > 15 else amparo,
            (resultados_clustering.loc[amparo, 'FRECUENCIA'],
             resultados_clustering.loc[amparo, 'SEVERIDAD_MEDIA']),
            fontsize=7,
            ha='center',
            va='bottom',
            alpha=0.8
        )
    
    sil = silhouette_scores[idx]
    ax.set_title(f'k = {k} clusters (Silhouette = {sil:.3f})', fontsize=12, fontweight='bold')
    ax.set_xlabel('Frecuencia')
    ax.set_ylabel('Severidad Media ($)')
    plt.colorbar(scatter, ax=ax, label='Cluster')

plt.suptitle('Clustering Jerárquico de Amparos\n(Tamaño = volumen de períodos)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## Resumen para Modelación de Credibilidad

In [ ]:
print("=" * 80)
print("RESUMEN COMPARATIVO DE CLUSTERS PARA MODELACIÓN")
print("=" * 80)

# Tabla resumen para cada k
for k in k_values:
    col_cluster = f'CLUSTER_k{k}'
    
    print(f"\n{'─'*70}")
    print(f"k = {k} (Silhouette = {silhouette_scores[k_values.index(k)]:.4f})")
    print(f"{'─'*70}")
    
    resumen = resultados_clustering.groupby(col_cluster).agg({
        'N_PERIODOS': 'sum',
        'N_SINIESTROS': 'sum',
        'FRECUENCIA': 'mean',
        'SEVERIDAD_MEDIA': 'mean',
        'EXPOSICION_TOTAL': 'sum',
        'PRIMA_PURA': 'mean'
    }).round(6)
    
    resumen['N_AMPAROS'] = resultados_clustering.groupby(col_cluster).size()
    resumen['PRIMA_PURA_CLUSTER'] = resumen['SEVERIDAD_TOTAL'] if 'SEVERIDAD_TOTAL' in resumen.columns else 0
    
    # Calcular prima pura real del cluster
    sev_total = resultados_clustering.groupby(col_cluster)['SEVERIDAD_TOTAL'].sum()
    exp_total = resultados_clustering.groupby(col_cluster)['EXPOSICION_TOTAL'].sum()
    resumen['PRIMA_PURA_REAL'] = (sev_total / exp_total).round(6)
    
    resumen = resumen[['N_AMPAROS', 'N_PERIODOS', 'N_SINIESTROS', 'FRECUENCIA', 
                       'SEVERIDAD_MEDIA', 'EXPOSICION_TOTAL', 'PRIMA_PURA_REAL']]
    resumen.columns = ['Amparos', 'Períodos', 'Siniestros', 'Freq Media', 
                       'Sev Media', 'Exposición', 'Prima Pura']
    
    print(resumen.to_string())

# Guardar asignación del mejor k en stats_amparo
stats_amparo['CLUSTER'] = cluster_assignments[best_k]

print(f"\n{'='*80}")
print(f"RECOMENDACIÓN: k = {best_k}")
print(f"{'='*80}")
print(f"""
El clustering jerárquico con k={best_k} tiene el mejor Silhouette Score ({best_score:.4f}).

Para la modelación de credibilidad, considerar:
- Clusters con suficientes datos (N_PERIODOS > 1,000)  
- Clusters con siniestralidad observable (N_SINIESTROS > 10)
- Homogeneidad en frecuencia y severidad dentro de cada cluster
""")

## Reasignación de Amparos por Clusters (k=4)

In [ ]:
print("=" * 80)
print("REASIGNACIÓN DE AMPAROS USANDO k=4 CLUSTERS")
print("=" * 80)

# Usar k=4
k_seleccionado = 4
col_cluster = f'CLUSTER_k{k_seleccionado}'

# Ver composición de cada cluster
print(f"\nComposición de clusters (k={k_seleccionado}):")
for cluster in range(k_seleccionado):
    mask = resultados_clustering[col_cluster] == cluster
    amparos = resultados_clustering[mask].index.tolist()
    print(f"\nCluster {cluster}: {amparos}")

# Crear mapeo de amparo original → nuevo amparo
mapeo_amparo = {}

for cluster in range(k_seleccionado):
    mask = resultados_clustering[col_cluster] == cluster
    amparos = resultados_clustering[mask].index.tolist()
    
    if cluster == 0:
        # Cluster 0 se llama "CALIDAD DE LA OBRA"
        nuevo_nombre = "CALIDAD DE LA OBRA"
    else:
        # Clusters 1, 2, 3: usar el nombre del único amparo
        if len(amparos) == 1:
            nuevo_nombre = amparos[0]
        else:
            # Si tiene más de uno, indicarlo
            print(f"⚠️  Cluster {cluster} tiene {len(amparos)} amparos: {amparos}")
            nuevo_nombre = amparos[0]  # Usar el primero por defecto
    
    for amparo in amparos:
        mapeo_amparo[amparo] = nuevo_nombre

print("\n" + "=" * 80)
print("MAPEO FINAL DE AMPAROS")
print("=" * 80)
for original, nuevo in sorted(mapeo_amparo.items()):
    if original != nuevo:
        print(f"  {original:40} → {nuevo}")
    else:
        print(f"  {original:40} (sin cambio)")

In [ ]:
# Aplicar mapeo a base_final_anual
print("=" * 80)
print("APLICANDO MAPEO A LOS DATASETS")
print("=" * 80)

# Cargar datasets originales
df_base = pd.read_csv('../data/tmp/base_final_anual.csv')
df_siniestros = pd.read_csv('../data/tmp/siniestros_consolidado.csv')

print(f"\nbase_final_anual.csv:")
print(f"  Registros: {len(df_base):,}")
print(f"  Amparos únicos antes: {df_base['AMPARO'].nunique()}")

print(f"\nsiniestros_consolidado.csv:")
print(f"  Registros: {len(df_siniestros):,}")
print(f"  Amparos únicos antes: {df_siniestros['AMPARO'].nunique()}")

# Aplicar mapeo
df_base['AMPARO_ORIGINAL'] = df_base['AMPARO']
df_base['AMPARO'] = df_base['AMPARO'].map(mapeo_amparo)

df_siniestros['AMPARO_ORIGINAL'] = df_siniestros['AMPARO']
df_siniestros['AMPARO'] = df_siniestros['AMPARO'].map(mapeo_amparo)

# Verificar
print(f"\n" + "-" * 40)
print("DESPUÉS DEL MAPEO:")
print("-" * 40)

print(f"\nbase_final_anual.csv:")
print(f"  Amparos únicos después: {df_base['AMPARO'].nunique()}")
print(f"  Distribución:")
print(df_base['AMPARO'].value_counts())

print(f"\nsiniestros_consolidado.csv:")
print(f"  Amparos únicos después: {df_siniestros['AMPARO'].nunique()}")
print(f"  Distribución:")
print(df_siniestros['AMPARO'].value_counts())

In [ ]:
# Guardar nuevos CSV
print("=" * 80)
print("GUARDANDO NUEVOS ARCHIVOS")
print("=" * 80)

# Guardar con nuevos nombres
df_base.to_csv('../data/tmp/base_final_anual_clustered.csv', index=False)
df_siniestros.to_csv('../data/tmp/siniestros_consolidado_clustered.csv', index=False)

print("\n✓ Archivos guardados:")
print("  → ../data/tmp/base_final_anual_clustered.csv")
print("  → ../data/tmp/siniestros_consolidado_clustered.csv")

print(f"\nResumen de nuevos amparos:")
print("-" * 50)
for amparo in df_base['AMPARO'].unique():
    n_registros = len(df_base[df_base['AMPARO'] == amparo])
    n_originales = df_base[df_base['AMPARO'] == amparo]['AMPARO_ORIGINAL'].nunique()
    print(f"  {amparo}: {n_registros:,} registros ({n_originales} amparos originales)")